In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
from urllib.request import Request, urlopen, urlparse
import re
import time
import requests
import validators

In [2]:
url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=end_date&seed=2743718&page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [3]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')

In [4]:
r.status_code

200

In [5]:
def engine(pages) : 
    url = url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=enddate&seed=2743718&page='
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    lst = []
    for i in range(1,pages) :
        r = requests.get(url+str(i),headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        lst.append(soup)
        i =+ 1   
    return lst

In [6]:
containers = soup.find_all('div', class_="js-react-proj-card")

In [7]:
def scraper_prototype(text_profile, prefix):

    if '"name"' in text_profile:
        text_profile = re.sub('"', '', text_profile)

    text2 = text_profile.split(',')
    dummy2 = [re.findall('[A-Za-z0-9_]+:', ww) for ww in text2]
    itemname = [dum_part[0] for dum_part in dummy2 if len(dum_part) > 0]
    ans_array = [re.search(xxy1+'(.*?)'+xxy2, text_profile).group(1).rstrip(',')
                 for xxy1, xxy2 in zip(itemname[:-1], itemname[1:])]
    itemnames = [x.rstrip(':') for x in itemname]

    info_dict = dict()
    for item, ans in zip(itemnames, ans_array):
        info_dict[prefix+'_'+item] = ans
    return info_dict

In [8]:
def to_string(container) : 
    whole_string = str(container).replace('&amp', '').replace('&quot;', '')
    return whole_string

In [9]:
def get_name(whole_string) : 
    if '"name"' in whole_string:
        project_info = '"name"' + re.findall('"name"(.+?)"creator"', whole_string)[0]
    else:
        project_info = 'name' + re.findall('name(.+?)creator', whole_string)[0]
    project_info_dict = scraper_prototype(project_info, 'project')
    df1 = pd.DataFrame.from_dict(project_info_dict, orient='index').transpose()
    return df1

In [10]:
def get_creator(whole_string) : 
    if '"creator"' in whole_string:
        creator_string = re.findall(r'\"creator\"\:\{(.+?)\}', whole_string)[0]
    else:
        creator_string = re.findall(r'creator\:\{(.+?)\}', whole_string)[0]
    creator_info_dict = scraper_prototype(creator_string, 'creator')
    creator_info_dict['creator_slug'] = creator_info_dict.get('creator_slug', 'Null')
    df2 = pd.DataFrame.from_dict(creator_info_dict, orient='index').transpose()
    
    return df2

In [11]:
def get_location(whole_string) : 
    if '"location"' in whole_string:
        location_string = re.findall(r'\"location\"\:\{(.+?)\}', whole_string)[0]
    else:
        location_string = re.findall(r'location\:\{(.+?)\}', whole_string)[0]
    location_info_dict = scraper_prototype(location_string, 'location')
    df3 = pd.DataFrame.from_dict(location_info_dict, orient='index').transpose()
    return df3

In [12]:
def get_profile(whole_string) : 
    if '"profile"' in whole_string:
        profile_string = re.findall(r'\"profile\"\:\{(.+?)\}', whole_string)[0]
    else:
        profile_string = re.findall(r'profile\:\{(.+?)\}', whole_string)[0]
    profile_info_dict = scraper_prototype(profile_string, 'profile')
    df4 = pd.DataFrame.from_dict(profile_info_dict, orient='index').transpose()
    return df4

In [13]:
def make_a_row(df1,df2,df3) : 
    df = pd.concat((df1, df2, df3), axis=1)
    return df

In [14]:
def row_flow(container) : 
    string = to_string(container)
    name = get_name(string)
    creator = get_creator(string)
    location = get_creator(string)
    #profile = get_profile(string)
    row = make_a_row(name,creator,location)
    return row

In [93]:
def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [20]:
row = row_flow(containers[0])
row0 = row_flow(containers[1])
row1 = row_flow(containers[2])
row2 = row_flow(containers[3])
row3 = row_flow(containers[4])
row4 = row_flow(containers[5])
row5 = row_flow(containers[6])
row6 = row_flow(containers[7])
row7 = row_flow(containers[8])
row8 = row_flow(containers[9])
row9 = row_flow(containers[10])
row10 = row_flow(containers[11])
len(row),len(row0),len(row1),len(row2),len(row3),len(row4),len(row5),len(row6),len(row7),len(row8),len(row9),len(row10)

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

In [100]:
df = pd.concat([row, row0,row2,row3,row4,row6,row7,row8,row10],ignore_index=True)

In [40]:
df

,project_name,project_blurb,project_goal,project_pledged,project_state,project_slug,project_disable_communication,project_country,project_country_displayable_name,project_currency,...,creator_small,creator_id,creator_name,creator_slug,creator_is_registered,creator_is_email_verified,creator_chosen_currency,creator_is_superbacker,creator_avatar,creator_small
0,If Fusion Were That Easy - point ; click comed...,\If Fusion Were That Easy\ is a humorous 2.5D ...,2000.0,2569.2,live,if-fusion-were-that-easy,false,DE,Germany,EUR,...,https://ksr-ugc.imgix.net/assets/034/478/113/b...,1973969641,Golden Coin Games,goldencoingames,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/034/47...,https://ksr-ugc.imgix.net/assets/034/478/113/b...
1,Meanwhile in Africa: Supplemental K-5 Social S...,Offering culturally responsive content with an...,5168.0,5178.0,live,meanwhile-in-africa-supplemental-k-5-social-st...,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/036/099/111/0...,194065216,Just Like Me Presents,meanwhileinafrica,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/036/09...,https://ksr-ugc.imgix.net/assets/036/099/111/0...
2,Black Silk Tarot Gold Foil Deck,An artist-illustrated tarot deck with beautifu...,5000.0,6515.0,live,black-silk-tarot-gold-foil-deck,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/034/696/931/4...,477708582,sara stocksdale,blacksilktarot,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/034/69...,https://ksr-ugc.imgix.net/assets/034/696/931/4...
3,Sea Witch Collection,A beautiful set of pins full of nautical magic,450.0,16282.0,live,sea-witch-collection,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/033/174/595/2...,2100073,Lily Brook Lane,lilybrooklane,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/033/17...,https://ksr-ugc.imgix.net/assets/033/174/595/2...
4,GHOSTS AND GHOULS AND OTHER CREEPY THINGS,Spine-tingling tales from NeoParadoxa featurin...,1500.0,5157.0,live,ghosts-and-ghouls-and-other-creepy-things,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/009/342/575/c...,2096699057,eSpec Books,e-specbooks,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/009/34...,https://ksr-ugc.imgix.net/assets/009/342/575/c...
5,ZOOT! : The Card Game for Word Nerds,ZOOT! is played with a deck of 118 colourful l...,5000.0,5541.29,live,zoot-the-card-game-for-word-nerds,false,CA,Canada,CAD,...,https://ksr-ugc.imgix.net/assets/036/149/328/c...,1659177433,Joan Krygsman,zootzoot,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/036/14...,https://ksr-ugc.imgix.net/assets/036/149/328/c...
6,Customizable Fantasy World and Region Maps for...,More than 40 world ; regional fantasy maps of ...,750.0,1844.29,live,customizable-fantasy-world-and-region-maps-for...,false,CA,Canada,CAD,...,https://ksr-ugc.imgix.net/assets/009/813/811/4...,1173384431,Jean Francois Senay,jfsenay,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/009/81...,https://ksr-ugc.imgix.net/assets/009/813/811/4...
7,Super Best Friend #1-2: A Superhero Adventure ...,The live-streamer sidekick of the world’s grea...,7500.0,15087.0,live,super-best-friend-1-2-a-superhero-adventure-comic,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/006/648/968/7...,827962922,Jason Inman,jawiin,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/006/64...,https://ksr-ugc.imgix.net/assets/006/648/968/7...
8,City of Spiritdale - A Fantasy City,"3D Printable STL Files, Terrain for Tabletop, ...",6300.0,142080.29,live,city-of-spiritdale-a-fantasy-city,false,CA,Canada,CAD,...,https://ksr-ugc.imgix.net/assets/035/236/461/6...,2145431169,TabletopTerrain,ian-margherit,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/035/23...,https://ksr-ugc.imgix.net/assets/035/236/461/6...


In [101]:
df = df.loc[:,~df.columns.duplicated()]

In [102]:
df.columns

Index(['project_name', 'project_blurb', 'project_goal', 'project_pledged',
       'project_state', 'project_slug', 'project_disable_communication',
       'project_country', 'project_country_displayable_name',
       'project_currency', 'project_currency_symbol',
       'project_currency_trailing_code', 'project_deadline',
       'project_state_changed_at', 'project_created_at', 'project_launched_at',
       'project_staff_pick', 'project_is_starrable', 'project_backers_count',
       'project_static_usd_rate', 'project_usd_pledged',
       'project_converted_pledged_amount', 'project_fx_rate',
       'project_usd_exchange_rate', 'project_current_currency', 'creator_id',
       'creator_name', 'creator_slug', 'creator_is_registered',
       'creator_is_email_verified', 'creator_chosen_currency',
       'creator_is_superbacker', 'creator_avatar', 'creator_small'],
      dtype='object')

In [163]:
site = []
lst = []
url = 'https://www.kickstarter.com/profile/'

In [181]:
for i in range(0,len(df)) : 
    html = requests.get('https://www.kickstarter.com/profile/'+str(df.loc[i]['creator_id'])+'/about', headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        df.iloc[i]['website'] = (a['href'])
        lst.append(a['href'])
        #site.append(df.iloc[i]['creator_id'])

In [87]:
df.drop('project_current_currency',axis=1,inplace=True)

/var/folders/q1/v8q8_chd1rd46t7djqglmp6r0000gn/T/ipykernel_31912/721596944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('project_current_currency',axis=1,inplace=True)


In [88]:
df.columns

Index(['project_name', 'project_blurb', 'project_goal', 'project_pledged',
       'project_state', 'project_slug', 'project_country', 'project_currency',
       'project_deadline', 'project_state_changed_at', 'project_created_at',
       'project_launched_at', 'project_usd_pledged',
       'project_converted_pledged_amount', 'project_fx_rate', 'creator_id',
       'creator_name', 'creator_slug', 'creator_avatar', 'creator_small'],
      dtype='object')

In [89]:
df

,project_name,project_blurb,project_goal,project_pledged,project_state,project_slug,project_country,project_currency,project_deadline,project_state_changed_at,project_created_at,project_launched_at,project_usd_pledged,project_converted_pledged_amount,project_fx_rate,creator_id,creator_name,creator_slug,creator_avatar,creator_small
0,If Fusion Were That Easy - point ; click comed...,\If Fusion Were That Easy\ is a humorous 2.5D ...,2000.0,2569.2,live,if-fusion-were-that-easy,DE,EUR,1646312865,1643720866,1628597968,1643720865,2867.676347544,2569,1.0,1973969641,Golden Coin Games,goldencoingames,{thumb:https://ksr-ugc.imgix.net/assets/034/47...,https://ksr-ugc.imgix.net/assets/034/478/113/b...
1,Meanwhile in Africa: Supplemental K-5 Social S...,Offering culturally responsive content with an...,5168.0,5178.0,live,meanwhile-in-africa-supplemental-k-5-social-st...,US,USD,1646313005,1643721006,1636989178,1643721005,5178.0,4665,0.901065,194065216,Just Like Me Presents,meanwhileinafrica,{thumb:https://ksr-ugc.imgix.net/assets/036/09...,https://ksr-ugc.imgix.net/assets/036/099/111/0...
2,Black Silk Tarot Gold Foil Deck,An artist-illustrated tarot deck with beautifu...,5000.0,6515.0,live,black-silk-tarot-gold-foil-deck,US,USD,1646314416,1643722419,1635281889,1643722416,6515.0,5870,0.901065,477708582,sara stocksdale,blacksilktarot,{thumb:https://ksr-ugc.imgix.net/assets/034/69...,https://ksr-ugc.imgix.net/assets/034/696/931/4...
3,Sea Witch Collection,A beautiful set of pins full of nautical magic,450.0,16282.0,live,sea-witch-collection,US,USD,1646316017,1643724019,1638976932,1643724017,16282.0,14671,0.901065,2100073,Lily Brook Lane,lilybrooklane,{thumb:https://ksr-ugc.imgix.net/assets/033/17...,https://ksr-ugc.imgix.net/assets/033/174/595/2...
4,GHOSTS AND GHOULS AND OTHER CREEPY THINGS,Spine-tingling tales from NeoParadoxa featurin...,1500.0,5157.0,live,ghosts-and-ghouls-and-other-creepy-things,US,USD,1646317067,1643725071,1636655045,1643725067,5157.0,4646,0.901065,2096699057,eSpec Books,e-specbooks,{thumb:https://ksr-ugc.imgix.net/assets/009/34...,https://ksr-ugc.imgix.net/assets/009/342/575/c...
5,ZOOT! : The Card Game for Word Nerds,ZOOT! is played with a deck of 118 colourful l...,5000.0,5541.29,live,zoot-the-card-game-for-word-nerds,CA,CAD,1646318100,1643726101,1604093595,1643726100,4350.4090387582,3951,0.71315281,1659177433,Joan Krygsman,zootzoot,{thumb:https://ksr-ugc.imgix.net/assets/036/14...,https://ksr-ugc.imgix.net/assets/036/149/328/c...
6,Customizable Fantasy World and Region Maps for...,More than 40 world ; regional fantasy maps of ...,750.0,1844.29,live,customizable-fantasy-world-and-region-maps-for...,CA,CAD,1646318805,1645109206,1642576264,1645109205,1450.0149726937,1315,0.71315281,1173384431,Jean Francois Senay,jfsenay,{thumb:https://ksr-ugc.imgix.net/assets/009/81...,https://ksr-ugc.imgix.net/assets/009/813/811/4...
7,Super Best Friend #1-2: A Superhero Adventure ...,The live-streamer sidekick of the world’s grea...,7500.0,15087.0,live,super-best-friend-1-2-a-superhero-adventure-comic,US,USD,1646319375,1643727377,1629226778,1643727375,15087.0,13594,0.901065,827962922,Jason Inman,jawiin,{thumb:https://ksr-ugc.imgix.net/assets/006/64...,https://ksr-ugc.imgix.net/assets/006/648/968/7...
8,City of Spiritdale - A Fantasy City,"3D Printable STL Files, Terrain for Tabletop, ...",6300.0,142080.29,live,city-of-spiritdale-a-fantasy-city,CA,CAD,1646319622,1643727624,1641391755,1643727622,111545.7552023782,101324,0.71315281,2145431169,TabletopTerrain,ian-margherit,{thumb:https://ksr-ugc.imgix.net/assets/035/23...,https://ksr-ugc.imgix.net/assets/035/236/461/6...


In [194]:
df = df.loc[:,~df.columns.duplicated()]

In [90]:
len(df)

9

In [94]:
for i in range(0, len(df)):
    print(df.iloc[i]['creator_id'])

1973969641
194065216
477708582
2100073
2096699057
1659177433
1173384431
827962922
2145431169


In [95]:
web = []
lst = []
url = 'https://www.kickstarter.com/profile/'

In [96]:
for i in range(0, len(df)) : 
    html = requests.get('https://www.kickstarter.com/profile/'+str(df.iloc[i]['creator_id'])+'/about', headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        site = a['href']
        df['website'] = site
        lst.append(a['href'])
        web.append(df.iloc[i]['creator_id'])

/var/folders/q1/v8q8_chd1rd46t7djqglmp6r0000gn/T/ipykernel_31912/2344242793.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['website'] = site


In [99]:
lst

['/?ref=nav',
 '/profile/goldencoingames',
 '/profile/goldencoingames',
 '/discover/advanced?backed=-1&ref=profile_location&state=live&woe_id=698064',
 '/profile/goldencoingames/about',
 '/profile/goldencoingames/created',
 '/profile/goldencoingames/comments',
 'http://goldencoingames.de/',
 'http://rat-run.de/',
 'https://twitter.com/GoldenCoinGame',
 'https://www.instagram.com/goldencoingame',
 '/?ref=nav',
 '/profile/meanwhileinafrica',
 '/profile/meanwhileinafrica',
 '/discover/advanced?backed=-1&ref=profile_location&state=live&woe_id=2389876',
 '/profile/meanwhileinafrica/about',
 '/profile/meanwhileinafrica',
 '/profile/meanwhileinafrica/created',
 'https://justlikemepresents.com',
 '/?ref=nav',
 '/profile/blacksilktarot',
 '/profile/blacksilktarot',
 '/discover/advanced?backed=-1&ref=profile_location&state=live&woe_id=2388929',
 '/profile/blacksilktarot/about',
 '/profile/blacksilktarot',
 '/profile/blacksilktarot/created',
 '/profile/blacksilktarot/comments',
 'https://sawasta.

In [110]:
for el in lst : 
    if is_valid(el) == True : 
        dodo.append(el)
    else : 
        pass

In [98]:
dodo

NameError: name 'dodo' is not defined

In [84]:
s = []

In [112]:
for el in dodo : 
    try :
        r = requests.get(el)
        t = str(r.text)
        shopify = t.find('shopify')
        s.append(shopify)
    except ConnectionError :
        pass

ConnectionError: HTTPSConnectionPool(host='www,thewaltzfilm.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x124d455b0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [117]:
s

[-1, -1, 362, -1, -1, -1, -1, -1, -1, 362, -1, -1, -1, -1, 362, -1, -1, -1, -1]

In [187]:
for i in range(0,len(containers)) : 
    string =  to_string(containers[i])
    name = get_name(string)
    creator = get_creator(string)
    location = get_location(string)
    profile = get_profile(string)
    my_dict[i] = name,creator,location,profile
    df = pd.DataFrame(my_dict)
    final_df = pd.concat([final_df, df],ignore_index=True)
    i =+ 1

In [240]:
for i in range(1, len(containers)) : 
    final_df = row_flow(containers[0])
    #next_row = row_flow(containers[i])
    final_df = pd.concat([final_df, row_flow(containers[i])],ignore_index=True)
    i =+1